# NOTES

# IMPORTS

In [1]:
import cv2
import imutils
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


# DETECTOR

In [2]:
# Load the face detector cascade and smile detector LeNet
cascade_path = 'C:/Users/Tajr/Desktop/Data/RadonPlus/RadonTechnology/Dev/Case Studi/smileDetector/cascade/haarcascade_fontalface_default.xml'
lenet_path = 'C:/Users/Tajr/Desktop/Data/RadonPlus/RadonTechnology/Dev/Case Studi/smileDetector/network/serialized/LeNet.hdf5'
detector = cv2.CascadeClassifier(cascade_path)
model = load_model(lenet_path)

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2450)             

In [4]:
model.weights

[<tf.Variable 'conv2d_1/kernel:0' shape=(5, 5, 1, 20) dtype=float32, numpy=
 array([[[[ 9.25237909e-02, -1.24955982e-01, -2.94484068e-02,
            3.43100727e-02,  1.89921230e-01,  7.74759576e-02,
           -5.41191595e-03, -7.14038685e-02, -3.29879439e-03,
            7.90266786e-03,  6.92921132e-02,  8.00896510e-02,
           -1.37387618e-01, -2.22235192e-02,  9.31745023e-02,
           -9.72425044e-02, -2.44507249e-02, -4.63780351e-02,
           -1.14981253e-02,  9.41582844e-02]],
 
         [[ 3.83387879e-02, -6.36295453e-02, -1.35485381e-02,
            7.17707872e-02,  2.62598433e-02,  8.37957337e-02,
            6.57510385e-02, -7.09246248e-02,  1.53807923e-02,
           -5.00195101e-02,  8.75080191e-03,  1.47496074e-01,
           -1.00918874e-01,  8.60038102e-02, -6.69865087e-02,
            1.01752849e-02, -5.31672277e-02,  5.92124127e-02,
           -3.64000835e-02,  4.55608182e-02]],
 
         [[ 3.51958089e-02,  2.66317353e-02, -1.87286079e-01,
            1.911631

# WEBCAM

In [5]:
video_path = 'C:/Users/Tajr/Desktop/Data/RadonPlus/RadonTechnology/Dev/Case Studi/smileDetector/video/iss.mp4'

# If a video path was not supplied, grab the reference to the web cam
# if not video_path:
camera = cv2.VideoCapture(0)
while True:
    (grabbed, frame) = camera.read()
    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frameClone = frame.copy()
    
    rects = detector.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    for(fX, fY, fW, fH) in rects:
        # extract roi of the face from the grayscale image, resize it to 28 x 28 and then prepare roi for classification
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # determine the probabilities of both smiling and not smiling then set the label accordingly
        (notSmiling, smiling) = model.predict(roi)[0]
        label = 'Smiling' if smiling > notSmiling else 'Not Smiling'

        # display the label and bound box rectangle on the output frame
        cv2.putText(frameClone, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY +fH), (0, 0, 255), 2)
    
    
    # show detected faces along with smiling/not smiling label
    cv2.imshow('Face', frameClone)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
camera.release()
cv2.destroyAllWindows()
